# Processor for Feedback from Tabbycat

In the administrator area of tabbycat, go to setup -> Export XML. Replace the filename 'Tournament.xml' in the next cell with your exported file.

Below, you will have to modify the questions to export to feedback.

In [ ]:
import xml.dom.minidom
Load_XML = xml.dom.minidom.parse('Tournament.xml')

In [ ]:
attr = Load_XML.getElementsByTagName('adjudicator')[0].attributes['name']

In [ ]:
attr.nodeValue

In [ ]:
from xml.dom.minicompat import NodeList
from xml.dom.minidom import Element
import itertools


def adj_to_feedback(adjs: NodeList[Element]) -> list[dict]:
    return [
            {
                'to': adj.attributes['name'].nodeValue,
                'score': fb.attributes['score'].nodeValue,
                'question': answer.attributes['question'].nodeValue,
                'answer': answer.childNodes[0].nodeValue
            }
            for adj in adjs for fb in adj.childNodes for answer in fb.childNodes 
         ] 

In [ ]:
import pandas as pd

In [ ]:
df_feedback = pd.DataFrame(adj_to_feedback(Load_XML.getElementsByTagName('adjudicator')))

In [ ]:
df_feedback

In [ ]:
# Note just above this cell which question (e.g. 'Q4') is the open feedback question you want to export and modify
# the next line accordingly.
df_feedback[df_feedback.question == 'Q4'].to_csv('feedback.csv')